In [1]:
!pip install geopandas
!rm -rf sample_data/
pip install censusgeocode

  Using cached geopandas-0.12.2-py3-none-any.whl (1.1 MB)
     ---------------------------------------- 5.1/5.1 MB 19.2 MB/s eta 0:00:00
     --------------------------------------- 22.0/22.0 MB 21.1 MB/s eta 0:00:00
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)


'rm' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
import geopandas as gpd
import json
import subprocess
import shutil
import os, glob
import time
import censusgeocode as cg
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Polygon, box

In [24]:
state = 'arkansas'
# Change county to the one you are sorting. Do not include "_County" as that is appended below.
county = 'Arkansas'

In [25]:
# Change "lavaugha" to your uark username; if your Box is stored in a different folder, you may need to go create that path instead
img_path               = "C:\\Users\\lavaugha\\Box\\Images\\states\\arkansas\\{}_County".format(county)
coord_path             = "G:\\My Drive\\states\\{}\\counties_filtered".format(state)

print("Paths:")
print("--Images           : " + img_path)
print("--Coordinates      : " + coord_path)

Paths:
--Images           : C:\Users\lavaugha\Box\Images\states\arkansas\Arkansas_County
--Coordinates      : G:\My Drive\states\arkansas\counties_filtered


In [26]:
def get_square_bounds(polygon):
    # Convert the input polygon to a shapely Polygon
    shapely_polygon = Polygon(polygon[0])
    
    # Get the bounding box of the polygon
    bbox = shapely_polygon.bounds
    
    # Extract the left, right, top and bottom coordinates
    left, bottom, right, top = bbox
    return f"{left} {right} {top} {bottom}"

In [18]:
# Open the file from which image coordinates will be retrieved given a task number.
print("Looking for coordinates in " + coord_path + "\\filtered_Arkansas_{}.geojson".format(county))
print("Pre-check: " + str(os.path.exists(coord_path + "\\filtered_Arkansas_{}.geojson".format(county))))
try:
    with open(coord_path + "\\filtered_Arkansas_{}.geojson".format(county), 'r') as f:
        if not f.closed:
            print("Opened coordinates file successfully")
        else:
            print("Did not open coordinates file successfully")
            sys.exit()
        img_coordinates_data = json.load(f)
        img_coordinates = gpd.GeoDataFrame.from_features(img_coordinates_data)
except:
    print("Did not open coordinates file successfully")
    traceback.print_exc()

# Extract the task number of an image and retrieve its corresponding coordinates
tasks_new = []
tasks_old = []
for image in tqdm(os.listdir(img_path)):
    img_rev = image[::-1]
    if "task" in image:
        task = (img_rev[img_rev.find('.')+1 : img_rev.find('k')])[::-1]
        latitude = str(img_coordinates_data['features'][int(task)]['geometry']['coordinates'][0][0][0])
        longitude = str(img_coordinates_data['features'][int(task)]['geometry']['coordinates'][0][0][1])
        geo_info = cg.coordinates(x=latitude, y=longitude)
        census_tract = str(geo_info['Census Tracts'][0]['TRACT'])
        block_group = geo_info['2020 Census Blocks'][0]['BLKGRP']
        
        # Categorize images
        if not os.path.exists(img_path + "\\" + census_tract + "_CensusTract\\" + block_group + "_BlockGroup"):
            os.makedirs(img_path + "\\" + census_tract + "_CensusTract\\" + block_group + "_BlockGroup")
        shutil.move(img_path + "\\" + image, img_path + "\\" + census_tract + "_CensusTract\\" + block_group + "_BlockGroup")
        tasks_new.append(task)

Looking for coordinates in G:\My Drive\states\arkansas\counties_filtered\filtered_Arkansas_Ashley.geojson
Pre-check: True
Opened coordinates file successfully
Looking for tracts in C:\Users\lavaugha\Downloads\AR_census_tracts_by_county\AR_census_tracts_Arkansas_County_format.json
Pre-check: True
Opened tracts file successfully
MultiPolygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
MultiPolygon
MultiPolygon
Polygon
Polygon
MultiPolygon
MultiPolygon
MultiPolygon
MultiPolygon


100%|██████████| 100/100 [00:53<00:00,  1.86it/s]
